In [17]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [18]:
##load the trained model, scaler pickle,onehot

model = load_model('model.h5')

#load tje encoder and scaler

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [19]:
#sample input data
input_data={
'CreditScore':600,
'Geography':'France',
'Gender':'Male',
'Age':40,
'Tenure':3,
'Balance':60000,
'NumOfProducts':2,
'HasCrCard':1,
'IsActiveMember':1,
'EstimatedSalary':50000
}

In [20]:
input_data

{'CreditScore': 600,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000}

In [21]:
#One hot encoding of Geography

geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\GenAIv2\ANN Classification\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [22]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [24]:
##Encode categorical variables
#label_encoder_gender=LabelEncoder()
##data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
#data


#Encode categorical values - Gender
input_df['Gender']=label_encoder_gender.fit_transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,40,3,60000,2,1,1,50000


In [27]:
#Combine one hot encoded columns with the input data
input_df= pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)

input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [28]:
#scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.47154541, -1.09997489,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [30]:
#predict the chrun
prediction = model.predict(input_scaled)
prediction


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


array([[0.08695529]], dtype=float32)

In [32]:
prediction_proba=prediction[0][0]
prediction_proba

0.08695529

In [34]:
if prediction_proba > 0.5:
    print("The customer is likely to chrun")
else:
    print("The cutomer is not likey to churn")


The cutomer is not likey to churn
